### Relative Affordability Calculaton

In [227]:
import pandas as pd
import numpy as np 
import geopy.distance
import pgeocode

pd.set_option('display.max_columns', None)

In [228]:
import warnings
warnings.filterwarnings('ignore')

In [229]:
zips_data = pd.read_csv('final_zip_data.csv')

In [230]:
zips_data.head(2)

,ZipCode,State,County,CBSA_Name,CountyFIPS,StateFIPS,Population,HouseholdsPerZipCode,WhitePopulation,BlackPopulation,HispanicPopulation,AsianPopulation,IndianPopulation,OtherPopulation,MalePopulation,FemalePopulation,AverageHouseValue,IncomePerHousehold,MedianAge,NumberOfBusinesses,over_65_ratio,school_rating,Hosp_count,Zip_count,HPI,Demand_score,Supply_score,listviews_vs_US,med_days_on_mkt,nielson_rank,UE_rate,Zillow_HVF,BEA_percap_income,Survey_Date,FIPS_State,FIPS_County,Region_Code,Division_Code,State_y,state_abbrev,County_Name,one_unit_buildings_rep,one_unit_units_rep,one_unit_value_reps,two_unit_buildings_rep,two_unit_units_rep,two_unit_value_rep,three_unit_building_rep,three_unit_untis_rep,three_unit_value_rep,five_unit_building_rep,five_unit_unit_rep,five_unit_value_rep
0,20001,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,38551,16500,13671,20682,3555,3147,488,1952,18912,19639,692900,116195,30.3,1978,0.078726,5.727273,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,7.1,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20002,DC,DISTRICT OF COLUMBIA,Washington Arlington Alexandria,1,11,52370,22973,17522,33303,2299,1312,596,999,24896,27474,637300,90424,34.9,1674,0.085151,4.851852,NaN,783.0,330.6,5.351171,70.234114,0.4816,41.0,7.0,4.4,8.2,86567.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [231]:
listings_data = pd.read_csv('redfin_cleaned_stage_1.csv')

In [232]:
listings_data.head(2)

,Unnamed: 0,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,zip
0,10730,MLS Listing,NaN,Condo/Co-op,810 O St NW #201,Washington,DC,20001.0,899900.0,2.0,2.0,OLD CITY II,1282.0,NaN,2019.0,250.0,702.0,972.0,Active,NaN,NaN,https://www.redfin.com/DC/Washington/810-O-St-...,BRIGHT MLS,DCDC514770,N,Y,38.908332,-77.023611,20001
1,147,MLS Listing,NaN,Townhouse,29 Q St NW,Washington,DC,20001,1175000.0,4.0,3.5,OLD CITY II,2313.0,1122.0,1900.0,8.0,508.0,NaN,Active,NaN,NaN,https://www.redfin.com/DC/Washington/29-Q-St-N...,BRIGHT MLS,DCDC514598,N,Y,38.911332,-77.011364,20001


In [233]:
smh_data = pd.read_csv('SMH_Projects.csv')
smh_data['MedianSalesPrice'] = (smh_data['MinBaseSalesPrice'] + smh_data['MaxBaseSalesPrice'])/2

In [234]:
print('There are',len(smh_data.Division.unique()),'SMH communities')

There are 13 SMH communities


In [235]:
price_dict = smh_data.groupby('MetroAreaID')['MedianSalesPrice'].mean().round().astype(int).to_dict()
smh_data['MetroIDAveragePrice'] = smh_data['MetroAreaID'].map(price_dict)

In [236]:
smh_data.Division.unique()

array(['Atlanta', 'Richmond', 'Aiken/Augusta', 'Charleston',
       'Northern Virginia', 'Columbia', 'Charlottesville', 'Maryland',
       'Raleigh', 'Charlotte', 'Orlando', 'Greenville/Spartanburg',
       'Wilmington'], dtype=object)

In [237]:
smh_data[smh_data['MetroAreaID'] == 1015].head(1)

,ProjectGroupID,AltProjectID,ProjectID,ElementName,Division,MetroAreaID,Latitude,Longitude,Address,city,State,state_fips,ZipCode,County,MetroAreaTitle,CountyName,county_fips,cbsa_code,cbsa_name,msa_name,ProductID,ProductDescription,Active,MinSquareFootage,MaxSquareFootage,MinBaseSalesPrice,MaxBaseSalesPrice,ProductTypeID,NumberBedrooms,NumberFullBaths,NumberHalfBaths,NumberGarageSpaces,MedianSalesPrice,MetroIDAveragePrice
80,APC,APC1,Avalon Park Wesley Chapel TH,Avalon Park Wesley Chapel TH (APC1),Orlando,1015,28.2323,-82.2711,5263 Little Stream Lane,Wesley Chapel FL,NaN,NaN,33545.0,57.0,Tampa,NaN,NaN,NaN,NaN,NaN,002111,Bellevue,Y,0,0,338452.0,338452.0,NaN,NaN,NaN,NaN,NaN,338452.0,338452


In [276]:
centroids = pd.read_csv('centroid.csv')
centroids.sort_values(by=['MetroAreaID'])

,MetroAreaID,Latitude,Longitude,Area
4,1000,38.805100,-77.047000,Northern Virginia
0,1001,52.229676,-21.012229,Maryland
5,1003,38.033554,-78.507980,Charlottesville
3,1004,35.884766,-78.625053,Raleigh
1,1006,37.541290,-77.434769,Richmond
2,1007,33.748783,-84.388168,Atlanta
11,1008,32.776566,-79.930923,Charleston
10,1010,34.000700,-81.034800,Columbia
6,1011,35.227085,-80.843124,Charlotte
9,1012,34.895900,-82.217200,Greenville/Spartanburg


In [277]:
#dict of centroid lat/long for each SMH community/metro area ID
centroids = pd.read_csv('centroid.csv')
centroids_dict = centroids.set_index('MetroAreaID').T.to_dict('list')

In [278]:
#lat/long and Median Sales Price for each SMH property
smh_geo = smh_data[['MetroAreaID', 'Latitude', 'Longitude', 'MedianSalesPrice', 'Division']]
smh_geo

,MetroAreaID,Latitude,Longitude,MedianSalesPrice,Division
0,1007,33.891702,-84.140783,389700.0,Atlanta
1,1006,37.616200,-77.521000,355390.0,Richmond
2,1013,33.593076,-81.824541,240900.0,Aiken/Augusta
3,1013,33.582816,-81.776370,212400.0,Aiken/Augusta
4,1013,33.582816,-81.776370,247900.0,Aiken/Augusta
...,...,...,...,...,...
411,1004,35.794500,-78.940100,483990.0,Raleigh
412,1012,34.725073,-82.222818,242900.0,Greenville/Spartanburg
413,1000,38.907833,-77.496722,866490.0,Northern Virginia
414,1000,38.797253,-77.652348,940490.0,Northern Virginia


In [279]:
centroids = centroids.sort_values(by='Area')
centroids.Area.to_list()

['Aiken/Augusta',
 'Atlanta',
 'Charleston',
 'Charlotte',
 'Charlottesville',
 'Columbia ',
 'Greenville/Spartanburg',
 'Maryland',
 'Northern Virginia',
 'Orlando',
 'Raleigh',
 'Richmond',
 'Wilmington ']

In [ ]:
#merge centroid table with smh_geo table & drop NAs
smh_centroid = smh_geo.merge(centroids, on='MetroAreaID', how="left")
smh_centroid = smh_centroid.drop('Area',axis=1)
smh_centroid.columns = ["MetroAreaID", "Latitude" ,"Longitude",'MedianSalesPrice','Area', "centroid_lat", "centroid_long"]
smh_centroid.dropna(inplace = True)
smh_centroid.head(3)

In [ ]:
smh_centroid['lat_long'] = list(zip(smh_centroid.Latitude,smh_centroid.Longitude))
smh_centroid['centroid'] = list(zip(smh_centroid.centroid_lat,smh_centroid.centroid_long))
smh_centroid = smh_centroid.drop(["Latitude", "Longitude", "centroid_lat", "centroid_long"], axis=1)
smh_centroid['distance'] = smh_centroid.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)

In [ ]:
#list of every SMH property and distance to centroid
smh_centroid.head(5)

In [423]:
## create a dataframe to test distance with redfin data

rf_distance = listings_data.drop(['URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
                                 'NEXT OPEN HOUSE END TIME','NEXT OPEN HOUSE START TIME','STATUS','SOLD DATE','Unnamed: 0',
                                 'FAVORITE','INTERESTED','MLS#'],axis=1)

rf_distance['STATE OR PROVINCE'] = rf_distance['STATE OR PROVINCE'].str.replace('fl','FL')
rf_distance['STATE OR PROVINCE'] = rf_distance['STATE OR PROVINCE'].str.replace('Fl','FL')
rf_distance['STATE OR PROVINCE'] = rf_distance['STATE OR PROVINCE'].str.replace('sc','SC')

In [424]:
rf_distance['lat_long'] = list(zip(rf_distance.LATITUDE,rf_distance.LONGITUDE))

In [425]:
rf_distance = rf_distance.drop(['LATITUDE','LONGITUDE','SOURCE'],axis=1)

In [426]:
rf_distance.head()

,SALE TYPE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,zip,lat_long
0,MLS Listing,Condo/Co-op,810 O St NW #201,Washington,DC,20001.0,899900.0,2.0,2.0,OLD CITY II,1282.0,NaN,2019.0,250.0,702.0,972.0,20001,"(38.9083318, -77.0236114)"
1,MLS Listing,Townhouse,29 Q St NW,Washington,DC,20001,1175000.0,4.0,3.5,OLD CITY II,2313.0,1122.0,1900.0,8.0,508.0,NaN,20001,"(38.911332, -77.011364)"
2,MLS Listing,Condo/Co-op,815 T St NW Unit A,Washington,DC,20001,825000.0,3.0,2.5,OLD CITY II,1577.0,NaN,1900.0,68.0,523.0,198.0,20001,"(38.9157505, -77.0236398)"
3,MLS Listing,Townhouse,2213 2nd St NW,Washington,DC,20001,835427.0,4.0,2.0,LEDROIT PARK,1584.0,1310.0,1922.0,58.0,527.0,NaN,20001,"(38.919624, -77.0146027)"
4,MLS Listing,Condo/Co-op,2914 11th St NW #103,Washington,DC,20001,650000.0,2.0,2.0,COLUMBIA HEIGHTS,1210.0,NaN,1956.0,75.0,537.0,365.0,20001,"(38.9271701, -77.0273233)"


## Inputs

#### Average Stanley Martin Base Price: 
- Average base price of all SMH homes in the selected SMH Community. 

In [427]:
#list of average sale price of each SMH Community 
smh = smh_data.groupby(['MetroAreaID']).agg({'MedianSalesPrice': ['mean']}).reset_index()
smh.columns = ['MetroAreaID', 'Average Sale Price']
smh.head()

,MetroAreaID,Average Sale Price
0,1000,700512.282609
1,1001,541461.428571
2,1003,492651.590909
3,1004,507030.000000
4,1006,342768.846154


#### Average Resale Price:
- Average resale price of the competitor houses within the radius of selected distance  from the SMH community. (Previous Year Closing price) 

In [428]:
validMetroAreas = ['Aiken/Augusta','Atlanta','Charleston','Charlotte','Charlottesville','Columbia ',
 'Greenville/Spartanburg','Maryland','Northern Virginia','Orlando','Raleigh','Richmond','Wilmington ']

In [429]:
metroCentroidGuide = smh_centroid.drop(['distance','lat_long'],axis=1)
metroCentroidGuide = metroCentroidGuide.drop_duplicates('Area')
metroCentroidGuide

,MetroAreaID,MedianSalesPrice,Area,centroid
0,1007,389700.0,Atlanta,"(33.748783, -84.388168)"
1,1006,355390.0,Richmond,"(37.54129, -77.434769)"
2,1013,240900.0,Aiken/Augusta,"(33.5076, -81.8604)"
5,1008,289900.0,Charleston,"(32.776566, -79.930923)"
6,1000,884990.0,Northern Virginia,"(38.8051, -77.047)"
8,1010,241900.0,Columbia,"(34.0007, -81.0348)"
13,1003,418600.0,Charlottesville,"(38.033554, -78.50798)"
17,1001,668490.0,Maryland,"(52.2296756, -21.0122287)"
22,1004,484490.0,Raleigh,"(35.884766, -78.625053)"
58,1011,408850.0,Charlotte,"(35.227085, -80.843124)"


In [440]:
# converts user input to km
radiusInput = input('Enter a radius in miles: ') 
radiusInput = float(radiusInput) * 1.60934
radiusInput = int(radiusInput)
# input metro area
metroAreaInput = input('Enter your metro area: ')
if metroAreaInput not in validMetroAreas :
    raise ValueError("Invalid metro area")

if metroAreaInput != 'Orlando' :
    rf_metro = rf_distance[rf_distance['STATE OR PROVINCE'] != 'FL']
else :  rf_metro = rf_distance[rf_distance['STATE OR PROVINCE'] == 'FL']

gotCentroid = metroCentroidGuide[metroCentroidGuide['Area'] == metroAreaInput]
gotCentroid = gotCentroid['centroid'].item()
rf_metro['centroid'] = [gotCentroid] * len(rf_metro)
rf_metro['distance'] = rf_metro.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)

# rf_distance = rf_distance[rf_distance['distance'] < radiusInput]
rf_metro['distance'] = rf_metro['distance'].astype(str)
rf_metro['distance'] = rf_metro['distance'].str.rstrip(' km')
rf_metro['distance'] = rf_metro['distance'].astype(float)
rf_metro['distance'] = rf_metro['distance'].round(2)
rf_metro = rf_metro[rf_metro['distance'] < radiusInput]
if len(rf_metro) < 1 :
    print('No homes in this radius, pick a larger radius')
elif len(rf_metro) > 1 : 
    rf_metroMeanPrice = round(rf_metro["PRICE"].mean())
    print(f'The mean home price in this area is ${rf_metroMeanPrice}')



Enter a radius in miles:  50 
Enter your metro area:  Charlottesville


The mean of prices in this area is $486257


In [199]:

# rf_distance['distance'] = rf_distance.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)
# rf_centroid.apply(lambda x: geopy.distance.geodesic(x.lat_long, x.centroid), axis = 1)

# in_radius = []

# for value in smh_geo_dict.values():

# if geopy.distance.vincenty(value, coords_2).km < 

#### Average Resale Year Built: 
- Average year when the resale homes were built. (Example 1992) 

## Calculations

##### Age of Resales: 
- Average Resale Year Built — Current Year (e.g., 2021 -1992 = 29) 

##### Expected Resale Premium: 
- Based on the age of Resales, this is derived from lookup table #1. Example: For  age band of 21-30, Resale Premium is 40% 

##### Actual Resale Premium: 
- (Average Stanley Martin Base Price — Average Resale Price)/ Average Resale Price 

##### Score: 
- No score if there is no Actual Resale Premium. 
- If Actual Resale Premium > Expected Resale  Premium, then 1. 
- Else, if Actual Resale Premium< Expected Resale Premium then 3, else 2. 

##### Results: 
- Is Pricing Above or Below the Expected Premium to Resales? 
- To get this Premium to Resales value is looked up to Resale Premium Analysis table. (Table 2) Relative Affordability Ranking: 
- Based on the Score from the Calculations above. Lookup Table #3 and obtain the ranking.


In [ ]:
ideas:
map, show radius , show score 
inputs: radius length, input address (come with community), 